In [1]:
!which python

/opt/anaconda3/envs/llama-env/bin/python


In [2]:
!python --version

Python 3.9.21


In [3]:
# Install llama-cpp-python with Apple GPU (Metal) support
!CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose

# Install Hugging Face Hub
!pip install huggingface_hub

Using pip 25.1 from /opt/anaconda3/envs/llama-env/lib/python3.9/site-packages/pip (python 3.9)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.4 MB/s eta 0:00:00a 0:00:01
  Running command pip subprocess to install build dependencies
  Using pip 25.1 from /opt/anaconda3/envs/llama-env/lib/python3.9/site-packages/pip (python 3.9)
    Obtaining dependency information for setuptools>=42 from https://files.pythonhosted.org/packages/58/29/93c53c098d301132196c3238c312825324740851d77a8500a2462c0fd888/setuptools-80.8.0-py3-none-any.whl.metadata
    Using cached setuptools-80.8.0-py3-none-any.whl.metadata (6.6 kB)
    Obtaining dependency information for scikit-build>=0.13 from https://files.pythonhosted.org/packages/c3/a3/21b519f58de90d684056c52ec4e45f744cfda7483f082dcc4dd18cc74a93/scikit_build-0.18.1-py3-none-any.whl.metadata
    Using cached scikit_build-0.18.1-py3-none-any.whl.metadata (18 kB)
    Obtaining dependency information for cmake>=3.18 from https://files.pythonhosted.or

In [4]:
from llama_cpp import Llama

In [5]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q2_K.bin"

In [6]:
from huggingface_hub import hf_hub_download

In [7]:
model_path = hf_hub_download(repo_id = model_name_or_path, filename = model_basename)

In [8]:
model_path

'/Users/atharva7/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q2_K.bin'

In [9]:
lcpp_llm = None

In [10]:
lcpp_llm = Llama(
    model_path = model_path,
    n_threads = 2,
    n_batch = 512,
    n_gpu_layers = 32
)

llama.cpp: loading model from /Users/atharva7/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q2_K.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 5120
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 40
llama_model_load_internal: n_head_kv  = 40
llama_model_load_internal: n_layer    = 40
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 5.0e-06
llama_model_load_internal: n_ff       = 13824
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 10 (mostly Q2_K)
llama_model_load_internal: model size = 13B
llama_model_load_internal: ggml ctx size =    0.11 MB
llama_model_load_

In [11]:
prompt = "Write a code for linear regression"

prompt_template = f''' SYSTEM: You are a helpful assistant and your task is to help the users with their questions in the prompt. answer in as much detail as possible

USER: {prompt}

ASSISTANT: 
'''

In [12]:
response = lcpp_llm(prompt = prompt_template, max_tokens = 512, temperature = 0.5, top_p = 0.95,
                    repeat_penalty = 1.2, top_k = 150, echo = True)


llama_print_timings:        load time =   724.85 ms
llama_print_timings:      sample time =   244.84 ms /   420 runs   (    0.58 ms per token,  1715.40 tokens per second)
llama_print_timings: prompt eval time =   724.77 ms /    51 tokens (   14.21 ms per token,    70.37 tokens per second)
llama_print_timings:        eval time = 27360.71 ms /   419 runs   (   65.30 ms per token,    15.31 tokens per second)
llama_print_timings:       total time = 28685.32 ms


In [13]:
lcpp_llm.params.n_gpu_layers

32

In [14]:
print(response)

{'id': 'cmpl-626e8f8c-c942-4003-8520-7763c8b8246b', 'object': 'text_completion', 'created': 1748296844, 'model': '/Users/atharva7/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q2_K.bin', 'choices': [{'text': ' SYSTEM: You are a helpful assistant and your task is to help the users with their questions in the prompt. answer in as much detail as possible\n\nUSER: Write a code for linear regression\n\nASSISTANT: \nOf course! Linear regression is a common statistical model used to predict a continuous outcome variable based on one or more predictor variables. Here\'s an example of how you could implement linear regression in Python using scikit-learn library:\n```\nfrom sklearn.linear_model import LinearRegression\nimport numpy as np\n\n# Generate some sample data\nX = np.random.normal(0, 1, size=(100, 5)) # X features\ny = np.random.normal(0, 1, size=100) # y outcome variable\n\n# Create a Linear Re

In [15]:
print(response["choices"][0]["text"])

 SYSTEM: You are a helpful assistant and your task is to help the users with their questions in the prompt. answer in as much detail as possible

USER: Write a code for linear regression

ASSISTANT: 
Of course! Linear regression is a common statistical model used to predict a continuous outcome variable based on one or more predictor variables. Here's an example of how you could implement linear regression in Python using scikit-learn library:
```
from sklearn.linear_model import LinearRegression
import numpy as np

# Generate some sample data
X = np.random.normal(0, 1, size=(100, 5)) # X features
y = np.random.normal(0, 1, size=100) # y outcome variable

# Create a Linear Regression object and fit the model
lr_model = LinearRegression()
X_train = X[:80] # train set
y_train = y[:80] # train labels
lr_model.fit(X_train, y_train)

# Predict on test set
X_test = X[80:] # test set
preds = lr_model.predict(X_test)

# Evaluate the model
print("Coefficients:", lr_model.coef_)
print("Standard 

### To deallocate the GPU cores we allocated to the model, we will delete the model instance.

In [16]:
del lcpp_llm

ggml_metal_free: deallocating


In [17]:
import gc
gc.collect()

451